In [1]:
import findspark
findspark.init('/home/akshay/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_tutorials_1").getOrCreate()

24/06/28 23:07:32 WARN Utils: Your hostname, akshay-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/28 23:07:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/28 23:07:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = [("tony","{'hair':'black','eyes':'blue'}")]

In [3]:
schema = ["name","props"]

In [4]:
df = spark.createDataFrame(data=data, schema=schema)

In [5]:
df.show(truncate=False)

+----+------------------------------+
|name|props                         |
+----+------------------------------+
|tony|{'hair':'black','eyes':'blue'}|
+----+------------------------------+



In [6]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import MapType, StringType

In [7]:
MapSchema = MapType(StringType(),StringType())

In [9]:
help(from_json)

Help on function from_json in module pyspark.sql.functions:

from_json(col: 'ColumnOrName', schema: Union[pyspark.sql.types.ArrayType, pyspark.sql.types.StructType, pyspark.sql.column.Column, str], options: Optional[Dict[str, str]] = None) -> pyspark.sql.column.Column
    Parses a column containing a JSON string into a :class:`MapType` with :class:`StringType`
    as keys type, :class:`StructType` or :class:`ArrayType` with
    the specified schema. Returns `null`, in the case of an unparseable string.
    
    .. versionadded:: 2.1.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    col : :class:`~pyspark.sql.Column` or str
        a column or column name in JSON format
    schema : :class:`DataType` or str
        a StructType, ArrayType of StructType or Python string literal with a DDL-formatted string
        to use when parsing the json column
    options : dict, optional
        options to control parsing. accepts the same o

In [10]:
df1 = df.withColumn("Mapprops", from_json("props",MapSchema))

In [12]:
df1.show(truncate=False)
df1.printSchema()

+----+------------------------------+-----------------------------+
|name|props                         |Mapprops                     |
+----+------------------------------+-----------------------------+
|tony|{'hair':'black','eyes':'blue'}|{hair -> black, eyes -> blue}|
+----+------------------------------+-----------------------------+

root
 |-- name: string (nullable = true)
 |-- props: string (nullable = true)
 |-- Mapprops: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [13]:
df2 = df1.withColumn("hair",df1.Mapprops.hair).withColumn("eyes",df1.Mapprops.eye)
df2.show()

+----+--------------------+--------------------+-----+----+
|name|               props|            Mapprops| hair|eyes|
+----+--------------------+--------------------+-----+----+
|tony|{'hair':'black','...|{hair -> black, e...|black|NULL|
+----+--------------------+--------------------+-----+----+

